Work on improving the tree traversel I had previously implemented.

In [1]:
%run ../scripts/notebook_settings_lean.py
import tskit

Loading ts and poplabels

In [2]:
ts = tskit.load("../results/All_Samples_relate/chromhapX.trees")

In [3]:
poplabels = pd.read_csv("../data/pops/haploid_x_8cluster.sample", sep=" ")

In [4]:
# Version which counts number of samples and notes when 50 % has coalesced.
def coalescence_ordering(tree, IDs, sample_counts):
    df_list = []
    for i in IDs:
        pop_list = []
        gen_list = []
        current_node = i
        while tree.depth(current_node) > 0:
            # Find parent node
            parent_node = tree.parent(current_node)
            c_samples = pd.Series([x for x in tree.samples(current_node)])
            c_sample_counts = c_samples.map(i_mapping).value_counts()
            # If pop is not already added to list, add pop and note coal time when it surpasses 50 %
            for p in c_sample_counts.index:
                if p  not in pop_list and c_sample_counts[p] > sample_counts[p]/2:
                    pop_list.append(p)
                    gen_list.append(tree.time(current_node))
            current_node = parent_node
        d = {"ID": i, "sites": tree.num_sites, "span": tree.span, "start": tree.interval[0]}
        for i in range(len(pop_list)):
            d["coal_{}".format(i)] = pop_list[i]
        for i in range(len(gen_list)):
            d["coal_date_{}".format(i)] = gen_list[i]
        df_list.append(pd.DataFrame(d, index=[i]))
    return pd.concat(df_list)

In [5]:
i_mapping = {}
for i, row in poplabels.iterrows():
    i_mapping[i] = row.GROUP
poplabel_counts = poplabels["GROUP"].value_counts().to_dict()
highest_ID = poplabels.index.max()+1

In [9]:
#%%timeit -n 10
c = 0
ID_list = list(i_mapping.keys())[10:12]
sample_counts = poplabels["GROUP"].value_counts()
for tree in ts.trees():
    if c >= 100:
        break
    df = coalescence_ordering(tree, ID_list, sample_counts)
    c += 1

In [15]:
%%timeit -n 10
c = 0
IDs = list(i_mapping.keys())[10:12]
for tree in ts.trees():
    if c >= 1:
        break
    c += 1
    df_list = []
    for i in IDs:
        pop_list = []
        gen_list = []
        current_node = i
        while tree.depth(current_node) > 0:
            # Find parent node
            parent_node = tree.parent(current_node)
            c_samples = pd.Series([x for x in tree.samples(current_node)])
            c_sample_counts = c_samples.map(i_mapping).value_counts()
            # If pop is not already added to list, add pop and note coal time when it surpasses 50 %
            for p in c_sample_counts.index:
                if p  not in pop_list and c_sample_counts[p] > sample_counts[p]/2:
                    pop_list.append(p)
                    gen_list.append(tree.time(current_node))
            current_node = parent_node

866 ms ± 61.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
for u in tree.samples():
    path = []
    v = u
    while v != tskit.NULL:
        path.append(v)
        v = tree.parent(v)
    print(u, "->", path)

182 -> [182, 564, 642]
178 -> [178, 563, 564, 642]
176 -> [176, 323, 563, 564, 642]
179 -> [179, 323, 563, 564, 642]
186 -> [186, 407, 487, 638, 641, 642]
188 -> [188, 407, 487, 638, 641, 642]
208 -> [208, 430, 487, 638, 641, 642]
194 -> [194, 375, 430, 487, 638, 641, 642]
204 -> [204, 375, 430, 487, 638, 641, 642]
1 -> [1, 517, 611, 631, 638, 641, 642]
17 -> [17, 411, 517, 611, 631, 638, 641, 642]
314 -> [314, 411, 517, 611, 631, 638, 641, 642]
130 -> [130, 379, 606, 611, 631, 638, 641, 642]
132 -> [132, 379, 606, 611, 631, 638, 641, 642]
112 -> [112, 520, 606, 611, 631, 638, 641, 642]
4 -> [4, 410, 520, 606, 611, 631, 638, 641, 642]
104 -> [104, 410, 520, 606, 611, 631, 638, 641, 642]
197 -> [197, 335, 616, 629, 631, 638, 641, 642]
236 -> [236, 335, 616, 629, 631, 638, 641, 642]
223 -> [223, 485, 616, 629, 631, 638, 641, 642]
243 -> [243, 464, 485, 616, 629, 631, 638, 641, 642]
230 -> [230, 364, 464, 485, 616, 629, 631, 638, 641, 642]
232 -> [232, 364, 464, 485, 616, 629, 631, 638, 6

In [112]:
%%timeit -n 1
sample_counts = poplabels["GROUP"].value_counts()
df_l = []
c = 0
for tree in ts.trees():
    if c >= 1000:
        break
    c += 1
    l_l = []
    l_c = []
    # Setup of base information
    for n in tree.timeasc()[:highest_ID]:
        l_l.append([n, tree.num_sites, tree.span, tree.interval[0]])
        l_c.append([])
    for n in tree.timeasc()[highest_ID:]:
        tree_time = tree.time(n)
        c_samples = (pd.Series([x for x in tree.samples(n)]))
        c_sample_counts = c_samples.map(i_mapping).value_counts()
        for p in c_sample_counts.index:
            if c_sample_counts[p] > sample_counts[p]/2:
                for i in c_samples:
                    if p not in l_l[i]:
                        l_l[i].append(p)
                        l_c[i].append(tree_time)
    for n in tree.timeasc()[:highest_ID]:
        l_l[n].extend(l_c[n])
    df_l.append(pd.DataFrame(l_l))

3min 19s ± 373 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [103]:
ts

In [113]:
114458*(3*60+19)/(60*60*1000)

6.326983888888889

Test using chr20 diploid and updated, array-based traversal

In [2]:
ts = tskit.load("../results/All_Samples_relate/chrom20.trees")

In [3]:
ts

In [4]:
poplabels = pd.read_csv("../steps/relate_All_Samples_prepared/chrom20.poplabels", sep=" ")

In [5]:
sample_counts = poplabels["GROUP"].value_counts()*2
i_mapping = {}
for i, row in poplabels.iterrows():
    i_mapping[i*2] = row.GROUP
    i_mapping[i*2+1] = row.GROUP
highest_ID = poplabels.index.max()*2+2

In [9]:
def coalescence_ordering(ts, highest_ID, sample_counts, i_mapping):
    df_l = []
    c = 0
    for tree in ts.trees():
        if c % 50 == 0:
            print(c)
        if c == 100:
            break
        c += 1
        l_l = []
        l_c = []
        # Setup of base information
        for n in tree.timeasc()[:highest_ID]:
            l_l.append([n, tree.num_sites, tree.span, tree.interval[0]])
            l_c.append([])
        for n in tree.timeasc()[highest_ID:]:
            tree_time = tree.time(n)
            c_samples = (pd.Series([x for x in tree.samples(n)]))
            c_sample_counts = c_samples.map(i_mapping).value_counts()
            for p in c_sample_counts.index:
                if c_sample_counts[p] > sample_counts[p]/2:
                    for i in c_samples:
                        if p not in l_l[i]:
                            l_l[i].append(p)
                            l_c[i].append(tree_time)
        for n in tree.timeasc()[:highest_ID]:
            l_l[n].extend(l_c[n])
        df_l.append(pd.DataFrame(l_l))
    return pd.concat(df_l)

In [10]:
print("Starting coalescence ordering with")
print("It has the following sample counts {}".format(sample_counts))
cols = ["ID", "sites", "span", "start"]
for i in range(len(sample_counts)):
    cols.append("coal_{}".format(i))
for i in range(len(sample_counts)):
    cols.append("coal_date_{}".format(i))

df = coalescence_ordering(ts, highest_ID, sample_counts, i_mapping)
df.columns = cols

Starting coalescence ordering with
It has the following sample counts GROUP
Tanzanian_Olive    134
Eastern_Yellow     104
Kindae              54
Hamadryas           52
Ethiopian_Olive     50
Papio               24
Western_Yellow      20
Chacma               8
Kenyan_Olive         4
Gelada               4
Name: count, dtype: int64
0
50
100


In [14]:
len(ts.trees())

201310

In [15]:
201310/5000

40.262

In [11]:
df

,ID,sites,span,start,coal_0,coal_1,coal_2,coal_3,coal_4,coal_5,...,coal_date_0,coal_date_1,coal_date_2,coal_date_3,coal_date_4,coal_date_5,coal_date_6,coal_date_7,coal_date_8,coal_date_9
0,0,2,31572.0,0.0,Tanzanian_Olive,Eastern_Yellow,Kindae,Hamadryas,Ethiopian_Olive,Western_Yellow,...,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,174536.796875
1,1,2,31572.0,0.0,Eastern_Yellow,Hamadryas,Western_Yellow,Tanzanian_Olive,Kindae,Ethiopian_Olive,...,20786.070312,20786.070312,28316.390625,49870.437500,49870.437500,49870.437500,49870.437500,49870.437500,49870.437500,174536.796875
2,2,2,31572.0,0.0,Tanzanian_Olive,Eastern_Yellow,Kindae,Hamadryas,Ethiopian_Olive,Western_Yellow,...,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,174536.796875
3,3,2,31572.0,0.0,Hamadryas,Eastern_Yellow,Western_Yellow,Tanzanian_Olive,Kindae,Ethiopian_Olive,...,18419.429688,20786.070312,28316.390625,49870.437500,49870.437500,49870.437500,49870.437500,49870.437500,49870.437500,174536.796875
4,4,2,31572.0,0.0,Tanzanian_Olive,Kenyan_Olive,Ethiopian_Olive,Chacma,Papio,Kindae,...,11536.129883,11536.129883,12514.884766,21653.517578,23218.597656,25539.082031,49870.437500,49870.437500,49870.437500,174536.796875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,449,24,538.0,56403.0,Papio,Kindae,Chacma,Eastern_Yellow,Tanzanian_Olive,Ethiopian_Olive,...,16779.443359,18908.380859,25866.656250,28517.242188,76487.125000,76487.125000,76487.125000,83242.109375,83242.109375,598308.250000
450,450,24,538.0,56403.0,Kindae,Papio,Chacma,Eastern_Yellow,Tanzanian_Olive,Ethiopian_Olive,...,23921.753906,23921.753906,25866.656250,28517.242188,76487.125000,76487.125000,76487.125000,83242.109375,83242.109375,598308.250000
451,451,24,538.0,56403.0,Kindae,Papio,Chacma,Eastern_Yellow,Tanzanian_Olive,Ethiopian_Olive,...,23921.753906,23921.753906,25866.656250,28517.242188,76487.125000,76487.125000,76487.125000,83242.109375,83242.109375,598308.250000
452,452,24,538.0,56403.0,Kindae,Papio,Chacma,Eastern_Yellow,Tanzanian_Olive,Ethiopian_Olive,...,23921.753906,23921.753906,25866.656250,28517.242188,76487.125000,76487.125000,76487.125000,83242.109375,83242.109375,598308.250000


In [8]:
%%timeit -n 1
df_l = []
c = 0
for tree in ts.trees():
    if c >= 100:
        break
    c += 1
    l_l = []
    l_c = []
    # Setup of base information
    for n in tree.timeasc()[:highest_ID]:
        l_l.append([n, tree.num_sites, tree.span, tree.interval[0]])
        l_c.append([])
    for n in tree.timeasc()[highest_ID:]:
        tree_time = tree.time(n)
        c_samples = (pd.Series([x for x in tree.samples(n)]))
        if len(c_samples) == 1:
            print(c_samples)
        c_sample_counts = c_samples.map(i_mapping).value_counts()
        for p in c_sample_counts.index:
            if c_sample_counts[p] > sample_counts[p]/2:
                for i in c_samples:
                    if p not in l_l[i]:
                        l_l[i].append(p)
                        l_c[i].append(tree_time)
    for n in tree.timeasc()[:highest_ID]:
        l_l[n].extend(l_c[n])
    df_l.append(pd.DataFrame(l_l))

33.2 s ± 362 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
cols = ["ID", "sites", "span", "start"]
for i in range(len(sample_counts)):
    cols.append("coal_{}".format(i))
for i in range(len(sample_counts)):
    cols.append("coal_date_{}".format(i))

In [58]:
concat_df = pd.concat(df_l)

In [59]:
concat_df.columns = cols

In [60]:
concat_df

,ID,sites,span,start,coal_0,coal_1,coal_2,coal_3,coal_4,coal_5,...,coal_date_0,coal_date_1,coal_date_2,coal_date_3,coal_date_4,coal_date_5,coal_date_6,coal_date_7,coal_date_8,coal_date_9
0,0,2,31572.0,0.0,Tanzanian_Olive,Eastern_Yellow,Kindae,Hamadryas,Ethiopian_Olive,Western_Yellow,...,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,174536.796875
1,1,2,31572.0,0.0,Eastern_Yellow,Hamadryas,Western_Yellow,Tanzanian_Olive,Kindae,Ethiopian_Olive,...,20786.070312,20786.070312,28316.390625,49870.437500,49870.437500,49870.437500,49870.437500,49870.437500,49870.437500,174536.796875
2,2,2,31572.0,0.0,Tanzanian_Olive,Eastern_Yellow,Kindae,Hamadryas,Ethiopian_Olive,Western_Yellow,...,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,64072.742188,174536.796875
3,3,2,31572.0,0.0,Hamadryas,Eastern_Yellow,Western_Yellow,Tanzanian_Olive,Kindae,Ethiopian_Olive,...,18419.429688,20786.070312,28316.390625,49870.437500,49870.437500,49870.437500,49870.437500,49870.437500,49870.437500,174536.796875
4,4,2,31572.0,0.0,Tanzanian_Olive,Kenyan_Olive,Ethiopian_Olive,Chacma,Papio,Kindae,...,11536.129883,11536.129883,12514.884766,21653.517578,23218.597656,25539.082031,49870.437500,49870.437500,49870.437500,174536.796875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,449,2,43.0,32372.0,Tanzanian_Olive,Kenyan_Olive,Ethiopian_Olive,Papio,Hamadryas,Eastern_Yellow,...,15262.021484,15262.021484,16441.609375,19872.951172,26645.255859,32689.662109,32689.662109,39295.140625,39295.140625,501990.625000
450,450,2,43.0,32372.0,Tanzanian_Olive,Eastern_Yellow,Kindae,Ethiopian_Olive,Hamadryas,Papio,...,46761.835938,46761.835938,46761.835938,46761.835938,46761.835938,46761.835938,46761.835938,46761.835938,46761.835938,501990.625000
451,451,2,43.0,32372.0,Western_Yellow,Tanzanian_Olive,Eastern_Yellow,Ethiopian_Olive,Kindae,Hamadryas,...,17012.982422,39295.140625,39295.140625,39295.140625,39295.140625,39295.140625,39295.140625,39295.140625,39295.140625,501990.625000
452,452,2,43.0,32372.0,Tanzanian_Olive,Eastern_Yellow,Kindae,Ethiopian_Olive,Hamadryas,Papio,...,46761.835938,46761.835938,46761.835938,46761.835938,46761.835938,46761.835938,46761.835938,46761.835938,46761.835938,501990.625000


In [63]:
highest_ID*len(sample_counts)/12619 # = 0.22 rough estimate of how many of these comparisons are essential.
# I could make a datastructure which remembers assignments, but I think it would increase complexity, not make it
# quicker

0.22965369680640305